In [5]:
from __future__ import print_function
import numpy as np
import six
import soundfile
import tensorflow.compat.v1 as tf

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim
import torch

/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/env/lib/python3.8/site-packages/tqdm-4.64.1-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
acoustic_feats = np.load("Judgenet/data/iemocap/features/acoustic_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy")
acoustic_feats.shape

(1, 128)

In [29]:
acoustic_feats = torch.load("data/features/audio/audio_2.pt")
# print(type(acoustic_feats[0][0]))
# print(acoustic_feats[0][0])

print(acoustic_feats.shape)
acoustic_feats

torch.Size([128])


tensor([ 83.5004, 108.3894, 212.7202, 176.8783, 142.0540, 153.5551, 179.3916,
         83.4707,  70.9247, 151.1612,  96.5224,  86.7620, 156.0555,  36.1285,
        104.5848, 106.4106, 114.1057, 187.6905,  75.1878, 130.4996, 185.2928,
        146.3186, 141.4350, 134.7932,  46.6471, 178.3863,  42.4091, 132.4144,
         85.3977,  97.4076, 118.3125,  80.4243,  67.4654,  77.8548, 172.7483,
         77.9947, 202.0015,  34.6190,  59.8335,  99.2198,  85.6000,  94.3331,
        175.9285, 143.4411, 166.7551, 168.5779, 130.0920, 101.1894,  99.2722,
        118.0038, 124.5490, 111.9909, 204.8281, 148.6821,  64.6144, 162.4312,
        136.8107, 120.2000, 187.4160, 146.5224,  95.6076,  34.1909, 114.9141,
        104.1802, 122.1741, 138.9757,  85.2091,  88.9445,  66.9338,  88.3536,
        187.7665, 150.0578, 135.8175, 176.6243, 178.6958, 164.0973, 126.8175,
        174.8494, 146.4875, 148.0342, 147.6548,  61.3080, 114.1262, 140.5627,
        112.9673, 124.2236, 117.1369, 135.3103,  88.2160, 135.55

In [28]:
acoustic_pooled = acoustic_feats.float()
print(acoustic_pooled)
acoustic_pooled = acoustic_pooled.mean(0)
acoustic_pooled.shape

tensor([[113.,  68., 178.,  ..., 103., 255., 107.],
        [ 57.,  75., 176.,  ..., 115.,  87., 201.],
        [ 59.,  85., 165.,  ..., 122., 136., 177.],
        ...,
        [151., 213., 255.,  ...,   0.,  62., 161.],
        [141., 158., 254.,  ...,   9., 174., 255.],
        [143., 217., 237.,  ...,   0.,   0.,  82.]])


torch.Size([128])

In [3]:
tf.app.flags.DEFINE_string('f', '', 'kernel')
flags = tf.app.flags

flags.DEFINE_string(
    'wav_file', "data/audio/0.wav",
    'Path to a wav file. Should contain signed 16-bit PCM samples. '
    'If none is provided, a synthetic sound is used.')

flags.DEFINE_string(
    'checkpoint', 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

flags.DEFINE_string(
    'pca_params', 'vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'tfrecord_file', None,
    'Path to a TFRecord file where embeddings will be written.')

FLAGS = flags.FLAGS

In [8]:
audioPath = "data/audio/0.wav"
examples_batch = vggish_input.wavfile_to_examples(audioPath)
examples_batch.shape

(1196, 96, 64)

In [9]:
pproc = vggish_postprocess.Postprocessor("vggish_pca_params.npz")
# writer = tf.python_io.TFRecordWriter("data/features/vggish")

In [10]:
with tf.Graph().as_default(), tf.Session() as sess:
    # Define the model in inference mode, load the checkpoint, and
    # locate input and output tensors.
    vggish_slim.define_vggish_slim(training=False)
    vggish_slim.load_vggish_slim_checkpoint(sess, 'vggish_model.ckpt')
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.OUTPUT_TENSOR_NAME)

    # Run inference and postprocessing.
    [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
    print(embedding_batch)
    postprocessed_batch = pproc.postprocess(embedding_batch)
    print(postprocessed_batch)

    # Write the postprocessed embeddings as a SequenceExample, in a similar
    # format as the features released in AudioSet. Each row of the batch of
    # embeddings corresponds to roughly a second of audio (96 10ms frames), and
    # the rows are written as a sequence of bytes-valued features, where each
    # feature value contains the 128 bytes of the whitened quantized embedding.
    seq_example = tf.train.SequenceExample(
        feature_lists=tf.train.FeatureLists(
            feature_list={
                vggish_params.AUDIO_EMBEDDING_FEATURE_NAME:
                    tf.train.FeatureList(
                        feature=[
                            tf.train.Feature(
                                bytes_list=tf.train.BytesList(
                                    value=[embedding.tobytes()]))
                            for embedding in postprocessed_batch
                        ]
                    )
            }
        )
    )
    print(seq_example)
    print(type(seq_example))
#     if writer:
#       writer.write(seq_example.SerializeToString())

#   if writer:
#     writer.close()

/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/env/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/env/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:332: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


INFO:tensorflow:Restoring parameters from vggish_model.ckpt
[[-0.16507158  0.34394336  0.2055729  ... -0.06952462  0.03148474
  -0.14506233]
 [-0.1151199   0.06069253  0.11505084 ... -0.3103042   0.23583683
  -0.1474416 ]
 [-0.12917903  0.1729928   0.09700073 ...  0.20218688  0.52634996
  -0.23428434]
 ...
 [-1.3246243   0.36623827 -0.05539152 ... -0.35059968  0.1463321
   0.32109934]
 [-1.6905849   0.08712962  0.04086894 ... -0.77959025 -0.04232813
   0.2320405 ]
 [-0.9782585   0.3818653  -0.02232465 ... -0.30936489  0.35436428
   0.09857889]]
[[113  74 152 ... 229 199 229]
 [ 59 127 164 ... 113 119 255]
 [ 40 109 135 ...   5 190 135]
 ...
 [139 255 250 ...   0 153  86]
 [112 255 255 ...   0 255 170]
 [141 205 255 ...  36 107 103]]
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "qJ\230\020\306\\{\255z\325\251\266\262\\odgh\213\324\200~\027W\224\201\241P\324FDr\201\215l(\253LiN\207\374x\215\000YA\005\226lE\3

AttributeError: shape

In [14]:
    print(postprocessed_batch.shape)


(1196, 128)
